In [2]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt

In [3]:
DATA_ROOT_DIR="FashionMnist"

In [5]:
train_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=True,
download=True,
transform=torchvision.transforms.ToTensor(),)
test_data=torchvision.datasets.FashionMNIST(
root=DATA_ROOT_DIR,
train=False,
download=True,
transform=torchvision.transforms.ToTensor(),)

In [6]:
BATCH_SIZE=64
train_data_loader=torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
test_data_loader=torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

In [12]:
type(train_data_loader)

torch.utils.data.dataloader.DataLoader

In [10]:
for X,y in test_data_loader:
    print(X.shape)
    print(y.shape, y.dtype, torch.unique(y))
    break

torch.Size([64, 1, 28, 28])
torch.Size([64]) torch.int64 tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [11]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [26]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten=nn.Flatten()
        self.linear_relu_stack=nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512,10),
        nn.ReLU(),
        )
    def forward(self, X): # forward propagation
        X=self.flatten(X)
        logits=self.linear_relu_stack(X)
        return logits

In [28]:
model=NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [29]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=1e-3)

In [34]:
"""def train(dataloader, model, loss_fn, optimizer):
    size=len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        X,y=X.to(device), y.to(device)
        # calculate error or loss value
        pred=model(X)
        loss=loss_fn(pred, y)
        
        # BackPropagation
        optimizer.zero_grad()
        loss.backword()
        optimizer.step()
        if batch %100==0:
            loss, current=loss.item(), batch*len(X)
            print(f"loss: {loss}[{current}/{size}]")"""

'def train(dataloader, model, loss_fn, optimizer):\n    size=len(dataloader.dataset)\n    for batch, (X,y) in enumerate(dataloader):\n        X,y=X.to(device), y.to(device)\n        # calculate error or loss value\n        pred=model(X)\n        loss=loss_fn(pred, y)\n        \n        # BackPropagation\n        optimizer.zero_grad()\n        loss.backword()\n        optimizer.step()\n        if batch %100==0:\n            loss, current=loss.item(), batch*len(X)\n            print(f"loss: {loss}[{current}/{size}]")'

In [35]:
def train(dataloader, model, loss_fn, optimzer):
  size = len(dataloader.dataset)

  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # calculate error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    optimzer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"training loss: {loss} [{current}/{size}]")

In [36]:
def test(dataloader, model):
    size=len(dataloader.dataset)
    model.eval()
    test_loss, correct=0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y=X.to(device), y.to(device)
            pred=model(X)
            test_loss=test_loss+ loss_fn(pred,y).item()
            correct=correct+(pred.argmax(1)==y).type(torch.float).sum().item()
        test_loss=test_loss/size
        correct=correct/size
        print(f"Test error: {100*correct}, average loss: {test_loss}")

In [38]:
EPOCHS=3
for epoch in range(EPOCHS):
    print(f"EPOCH: {epoch +1}\n----------")
    train(train_data_loader, model, loss_fn, optimizer)
    test(test_data_loader,model)
print("done")

EPOCH: 1
----------
training loss: 2.095754861831665 [0/60000]
training loss: 2.040386915206909 [6400/60000]
training loss: 1.9379444122314453 [12800/60000]
training loss: 1.99297297000885 [19200/60000]
training loss: 1.9477033615112305 [25600/60000]
training loss: 2.0026211738586426 [32000/60000]
training loss: 2.012639284133911 [38400/60000]
training loss: 1.9635835886001587 [44800/60000]
training loss: 2.0200419425964355 [51200/60000]
training loss: 2.0333521366119385 [57600/60000]
Test error: 39.989999999999995, average loss: 0.03020854287147522
EPOCH: 2
----------
training loss: 1.9798823595046997 [0/60000]
training loss: 1.9020881652832031 [6400/60000]
training loss: 1.7650812864303589 [12800/60000]
training loss: 1.8536791801452637 [19200/60000]
training loss: 1.8026074171066284 [25600/60000]
training loss: 1.8916456699371338 [32000/60000]
training loss: 1.9051045179367065 [38400/60000]
training loss: 1.85727858543396 [44800/60000]
training loss: 1.9294841289520264 [51200/60000]